In [8]:
#imports
import cv2
from imutils import paths
import numpy as np
import math
import pandas as pd

In [9]:
def vid2frame(filepath,strip_size,strip_x_location,output_frame_length,frame_name_start = 0,frames_number = -1):
    
    capture = cv2.VideoCapture(filepath)
    frameNr = 0
    output_frameNr = frame_name_start
    
    output_frame_strips = math.ceil(output_frame_length/strip_size)         #Number of strips required to make one output frame

    # Create OpenCV objects
    bgsub = cv2.createBackgroundSubtractorMOG2()

    #Create ROI mask
    rectangle = np.zeros((512, 512), dtype="uint8")
    cv2.rectangle(rectangle, (150, 512), (350, 0), 255, -1)

    #List to store generated file names
    dataNames = []

    while (True):

        success, frame = capture.read()

        if success and ( (frameNr <frames_number + output_frame_strips) or frames_number == -1  ):

            #backsub_frame = bgsub.apply(frame)  # apply MOG2 background subtraction    
            #skipped noise reduction
            #th, bin_frame = cv2.threshold(backsub_frame, 128, 255, cv2.THRESH_BINARY)  #Binarise frame
            #masked_frame = cv2.bitwise_and(rectangle, frame)           # Apply mask to frame
            rotate_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)          # Rotate frame
            crop_frame = rotate_frame[150:350, :]

            strip = crop_frame[:, strip_x_location:strip_x_location +strip_size]

            if frameNr == 0 :
                final_frame = strip
            else :
                final_frame = np.concatenate((final_frame,strip),axis=1)
                
            if(len(final_frame[0]) == output_frame_length):         # When desired output length is reached, start removing tyhe first strip
                
                #save_path = 
                name = 'frame_'+str(output_frameNr)+'.jpg'
                cv2.imwrite(f'data/images/bubbly/{name}', final_frame)
                dataNames.append(name) 
                output_frameNr = output_frameNr + 1 # change back to 1
                final_frame = final_frame[:,strip_size:]
                
        else:
            break

        frameNr = frameNr+1
    capture.release()

    return output_frameNr , dataNames
    

In [14]:
def getFileLabels(path,dataNames):
    
    video_name = path.split('/')[-1]
    label = video_name.split('_')
    t = int(label[2].split('=')[-1])
    m = int(label[3].split('=')[-1])
    x = float(label[4].split('=')[-1])
    q = label[5].split('=')[-1]

    data = {'filename': dataNames,
        't': t,
        'm': m,
        'x': x,
        'q': q}

    return pd.DataFrame(data)
    

In [11]:
def createCSV(datasets):
    csv = datasets[0]

    for i in range(1,len(datasets)):
        csv  =  csv.append(datasets[i])

    csv.to_csv('data/data_labels.csv',index=False)

    

In [21]:
path = r'D:\Flow Videos\regime_videos_2\regime_videos\\'

#Parameters
strip_size = 1        # Width of strip in pixels
frames_req =  2500 - 1   # number of frames used (generates 1 extra hence -1)
strip_x_location = 1     # Horizontal loaction of strip start in pixels
output_frame_length = 1000  # Size of Output frame in pixels


regime_type_paths = os.listdir(path)
regime_types = [ i.split('_')[-1] for i in regime_type_paths ]
print(regime_types)
path = path + regime_type_paths[0]
video_names = os.listdir(path)
video_paths = [ (path +'/'+ i) for i in video_names ]

video_paths



['bubbly', 'bubbly-slug', 'slug', 'slug-churn', 'churn', 'churn-annular', 'annular', 'mist']


['D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20201211_15.00_T=-25_mf=15_x=-0.0037_q=AD_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20201211_15.26_T=-25_mf=15_x=0.0049_q=AD_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20201215_16.41_T=-25_mf=10_x=0.0112_q=AD_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20210120_17.03_T=+5_m=23_x=-0.0061_q=AD_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20210127_15.50_T=+5_m=5_x=-0.0166_q=AD_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20210128_15.58_T=+5_m=5_x=-0.0083_q=5000_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20210128_16.25_T=+5_m=5_x=-0.0073_q=5000_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\\\1_bubbly/20210128_16.56_T=+5_m=5_x=0.0048_q=5000_up_bubbly.mp4',
 'D:\\Flow Videos\\regime_videos_2\\regime_videos\\

In [15]:

frames_gen = 0
frames_left = frames_req

datasets = []

for i in video_paths:

    if frames_gen < frames_req :
        frames_gen,dataNames = vid2frame(i,strip_size,strip_x_location,output_frame_length,frames_gen,frames_left)
        datasets.append(getFileLabels(i,dataNames))
        print('gen',frames_gen)
        frames_left = frames_req - frames_gen
        print('req',frames_left)
    else:
        break

createCSV(datasets)
print('CSV Created')



gen 1185
req 1314
gen 2370
req 129
gen 2500
req -1
CSV Created
